In [ ]:
# Calculated the volume on the basis of eccentricity of fitted ellipse

In [26]:
import pandas as pd
import math
import re

df = pd.read_csv('../Data/fitted_ellipse_parameter.csv')

def calculate_volume_in_small_e(semi_major_axis, semi_minor_axis):
    a = semi_major_axis
    b = semi_minor_axis
    radius = (a + b)/2
    return (4/3) * math.pi * (radius ** 3)

def calculate_volume_in_large_e(semi_major_axis, semi_minor_axis):
    a = semi_major_axis
    b = semi_minor_axis
    return (4/3) * math.pi * (a ** 2) * b

def pixels_to_mm(x):
    scale_factor = 0.083
    return scale_factor * x

def mm3_to_ml(volume_mm3):
    volume_ml = volume_mm3 / 1000
    return volume_ml

def get_numerical_part_from_label(apple_label):
    return re.match(r'^\d+', apple_label).group()
    
def get_apple_avg_volume():
    for key, value in apple_volume_map.items():
        average = sum(value) / len(value)
        apple_volume_map[key] = round(average, 2)
    
# 2530 px ~ 210 mm
# scale factor 0.083
apple_volume_map = {}
predicted_volume = []
for index, row in df.iterrows():
    semi_major_axis = pixels_to_mm(row['semi_major_axis'])
    semi_minor_axis = pixels_to_mm(row['semi_minor_axis'])
    apple_label = row['apple_label']
    if row['eccentricity'] <= 0.3:
        volume = calculate_volume_in_small_e(semi_major_axis, semi_minor_axis)
    else:
        volume = calculate_volume_in_large_e(semi_major_axis, semi_minor_axis)

    volume = mm3_to_ml(volume)
    volume = round(volume, 2)
    apple_volume_map.setdefault(get_numerical_part_from_label(apple_label), []).append(volume)
    predicted_volume_instance = [row['apple_label'],volume]
    predicted_volume.append(predicted_volume_instance)
    
get_apple_avg_volume()
column_names = ['apple_label', 'predicted_volume']
df = pd.DataFrame(predicted_volume, columns=column_names)
df.to_csv('../Data/predicted_volume_per_view.csv', index=False)
apple_volume_list = [[k, v] for k, v in apple_volume_map.items()]
df = pd.DataFrame(apple_volume_list, columns=column_names)
df.to_csv('../Data/predicted_volume.csv', index=False)